In [58]:
##Variables

import random 

POPULATION_SIZE = 200
NUM_GENERATIONS = 50

MUTATION_RATE = 0.1

# Define mutation function


allowed_notes = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8']

In [59]:
data = []
with open("./data/choplin.txt", "r") as dataFile:
    lines = dataFile.readlines()
    for l in lines:
        data.append(l.strip().replace(".",""))
data[:10]

data = data[:1000]

In [60]:
##Functions!!!

def levenshtein_distance(s, t):
    """
    Calculate the Levenshtein distance between two strings s and t.
    """
    m, n = len(s), len(t)
    d = [[0] * (n+1) for _ in range(m+1)]
    for i in range(m+1):
        d[i][0] = i
    for j in range(n+1):
        d[0][j] = j
    for j in range(1, n+1):
        for i in range(1, m+1):
            if s[i-1] == t[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = 1 + min(d[i-1][j], d[i][j-1], d[i-1][j-1])
    return d[m][n]

def string_similarity(s1, s2):
    """
    Calculate the similarity between two strings s1 and s2 using the Levenshtein distance algorithm.
    The similarity is the inverse of the Levenshtein distance normalized by the length of the longer string.
    """
    distance = levenshtein_distance(s1, s2)
    max_len = max(len(s1), len(s2))
    min_len = min(len(s1), len(s2))
    similarity = 1 - (distance / max_len)
    return similarity

def checker(s1):
    max_score = 0
    for thing in data:
        score = string_similarity(s1, thing)
        if score > max_score:
            max_score=score
    return max_score

def checkerV2(s1):
    all_data = "".join(data)
    return string_similarity(s1, all_data)

###ALL of the above is the Fitness! Jonahs Fitness!!!


def crossover(parent1, parent2):
    midpoint = random.randint(0, len(parent1))
 
    child = parent1[:midpoint] + parent2[midpoint:]
    return str(child)



def mutate(s1):
    notes = list(s1)
    for i in range(len(notes)//2):
        if random.random() < MUTATION_RATE:
            rand_choice = randomInRange(len(allowed_notes))
            notes[i*2] = allowed_notes[rand_choice][0]
            notes[i*2+1] = allowed_notes[rand_choice][1]
    return "".join(notes)


# Main genetic algorithm loop

def randomNoteSize():
    return random.randint(3,7)

def randomInRange(upper):
    return random.randint(0, upper-1)
    
def makeInitialPopulation():
    init_population = []
    for i in range(POPULATION_SIZE):
        temp = ""
        for j in range(randomNoteSize()):
            chord = randomInRange(len(data))
            note = randomInRange(len(data[chord])//2)
         
            noteSym = data[chord][(note)*2:note*2+2]
            temp = temp + noteSym
        init_population.append(temp)
        temp = []

    return init_population

In [ ]:
##Step 1 - Make initial Population

init_population = makeInitialPopulation()

population = init_population
for generation in range(NUM_GENERATIONS):
    print("Generation: " + str(generation))

    
    # Calculate fitness scores for each individual in population
    fitness_scores = [checkerV2(individual) for individual in population]
    
    parents = population
    for i in range(POPULATION_SIZE):
        for j in range(POPULATION_SIZE):
            if fitness_scores[i] > fitness_scores[j]:
                tempr = fitness_scores[i]
                fitness_scores[i] = fitness_scores[j]
                fitness_scores[j] = tempr                
                tempPar = parents[i]
                parents[i] = parents[j]
            
                parents[j] = tempPar
                
                
                
    # Select top individuals to be parents for next generation

    # Generate next generation through crossover and mutation
    next_gen = parents[:POPULATION_SIZE//2]

    while len(next_gen) < POPULATION_SIZE:
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
      
        child = crossover(parent1, parent2)
    
        child = mutate(child)
       
        next_gen.append(child)
    
    # Update population for next generation
    population = next_gen[:]

# Print final best individual

Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
Generation: 10
Generation: 11
Generation: 12
Generation: 13
Generation: 14
Generation: 15
Generation: 16
Generation: 17
Generation: 18
Generation: 19
Generation: 20
Generation: 21
Generation: 22
Generation: 23
Generation: 24
Generation: 25
Generation: 26
Generation: 27
Generation: 28


In [ ]:
final_pop = []

for note in population:
    final_text=""
    for i in range(len(note)):
        final_text+= note[i]
        if (i+1)%2 == 0:
            final_text+= '.'
            
    final_pop.append(final_text[:-1])
    

    
final_pop



In [ ]:
import music21
def save_to_midi(notes_list,filename):
    '''
    notes: list of str of notes separated by dots 
    example of filename: 'output12.mid'
    '''

    # Create a Stream object and a Part object
    stream = music21.stream.Stream()
    part = music21.stream.Part()

    # Add each note to the Part object
    for notes in notes_list:
        note_list = notes.split('.')
        for note in note_list:
            if note == '':
                pass
            else:
                n = music21.note.Note(note)
                part.append(n)

    # Add the part to the stream and write to MIDI file
    stream.append(part)
    mf = music21.midi.translate.streamToMidiFile(stream)
    mf.open(filename, 'wb')
    mf.write()
    mf.close()
    
save_to_midi(final_pop, "NewFitClassical_gen_50.midi")